In [2]:
library('dplyr')
library("lubridate")
directory = '/media/nicolas/Elements/data/'
time_past = 1990
time_pre = 2020
var <- list('TG','TX','TN','RR')

In [3]:
openfile <- function(path_to_data,time){
    
    if (!file.exists(path_to_data)){
            stop("Invalid ID")
    }
    
    Data <- read.table(path_to_data,skip = 20,header=TRUE,sep=',')
    Data <- filter(Data,Data[,5]==0)               
    Data <- Data[,-c(1,2,5)]
    Data$DATE <- ymd(Data$DATE)
 
    Data <- filter ( Data, year(Data$DATE) > time - 5 & year(Data$DATE) <= time)
    # Data <- filter ( Data, year(Data$DATE) == time)

    if (!length(Data$DATE)==1827){                              # 365 * 5 = 1825 Mais attention aux années bissextiles !
    
        stop('Unvailable data for this period')
    }
    return(Data)
}

In [34]:
STAID = List_sta[1,]
path_to_data_min <- paste(directory,'TN/TN_STAID',STAID,'.txt',sep='')
path_to_data_max <- paste(directory,'TX/TX_STAID',STAID,'.txt',sep='')
path_to_temp = paste(directory,'TG/TG_STAID',STAID,'.txt',sep='')
path_to_prec = paste(directory,'RR/RR_STAID',STAID,'.txt',sep='')
length(openfile(path_to_data_max,time_pre)[,1])

[1] 1827

In [4]:
Wichsta <- function(variables_list){
## Renvoie une liste de character correspondant aux IDs de stations pour lesquels des données 
## sont disponibles dans tous les dossiers de variables_list
    
    # list_comm <-NULL
    for (i in 1:length(variables_list)){
        sources=read.csv(file =paste(directory,variables_list[i],"/stations.txt",sep=''),sep=",", 
                   skip = 17,header = TRUE)
        sources$STAID<-as.integer(sources$STAID)
        if (i==1){list_comm<-sources['STAID']}
        else{list_comm<-intersect(list_comm,sources['STAID'])}
        }
    list_comm$STAID<-formatC(list_comm$STAID, width = 6, format = "d", flag = "0")
    return(list_comm)
}



mergedata <- function(variables_list){
## Renvoie une liste de character correspondant au ID de stations pour lesquels des données 
## sont disponibles entre le 1 janvier 1980 et le 31 décembre 2020
    
    list_comm <-NULL   #INITIALISATION
    
    for (i in 1:length(variables_list)){ 
        
    #Ouverture du fichier sources        
        sources=read.csv(file =paste(directory,variables_list[i],"/sources.txt",sep=''),sep=",", 
                   skip = 23,header = TRUE)
    #Convertissement ( Oui on dit comme ça j'ai ggler ) de la classe 
    #des colonnes START/STOP en dttm 
        sources$START <- ymd(sources$START)
        sources$STOP <- ymd(sources$STOP)
        sources=filter(sources,year(sources$STOP) >= time_pre   &    year(sources$START) <= time_past-5 )
        sources$STAID<-as.integer(sources$STAID)
    #Première itération : toutes les stations sont ajoutées à list_comm   
        if (i==1){list_comm<-sources['STAID']}
    #Suite : Les stations présentes dans list_comm et dans sources$STAID sont entrées dans list_comm
        else{list_comm<-intersect(list_comm,sources['STAID'])}
        }
    #Convertissement des valeurs de list_comm en un format exploitable pour ouvrir des fichiers
    list_comm$STAID<-formatC(list_comm$STAID, width = 6, format = "d", flag = "0")
    return(list_comm)
} 

In [5]:
allBIO <- function(STAID,time){
    
    if (!time == time_pre) {
        if(!time == time_past){
        stop('Time argument must be time_pre or time_past')
    }}
    
    prec_byquarter <- NULL
    temp_byquarter <- NULL
    
    path_to_data_min <- paste(directory,'TN/TN_STAID',STAID,'.txt',sep='')
    path_to_data_max <- paste(directory,'TX/TX_STAID',STAID,'.txt',sep='')
    path_to_temp = paste(directory,'TG/TG_STAID',STAID,'.txt',sep='')
    path_to_prec = paste(directory,'RR/RR_STAID',STAID,'.txt',sep='')
    
    data_temp <- openfile(path_to_temp,time)
    data_prec <- openfile(path_to_prec,time)
    
    #On extrait de la colonne de date un vecteur ne contenant que les années
    vec_year<-as.numeric(format(data_temp$DATE,"%Y"))
    #On extrait de la colonne de date un vecteur ne contenant que les mois
    vec_month<-as.numeric(format(data_temp$DATE, "%m"))
    
    data_temp_annual_mean<-aggregate(data_temp$TG,by = list(vec_year),mean)
    data_temp_annual_sd<-aggregate(data_temp$TG,by = list(vec_year),sd)
    
    #Calcule de la température moyenne de tout les mois de toutes les année : 
    data_temp_monthly_mean<-aggregate(data_temp$TG,by = list(vec_month),mean)
    
    BIO1 = mean (data_temp_annual_mean$x)
    BIO3 = mean (data_temp_annual_sd$x)
    BIO4 = sd (data_temp_monthly_mean$x)
    
    #Pluie cumulée tous les ans :
    Precipitation_annual<-aggregate(data_prec$RR,by = list(vec_year),sum)
    
    #Calcule de la pluie cumulée de tout les mois de toutes les année :
    data_prec_annual_monthly_sum<-aggregate(data_prec$RR,by = list(vec_year,vec_month),sum)
    
    #Moyenne sur les 5 ans <- pas d'effet 'année exeptionnel' 
    data_prec_monthly_sum <- aggregate(data_prec_annual_monthly_sum$x,
                                   data_prec_annual_monthly_sum[2],mean)
    
    sd<-sd(data_prec_monthly_sum$x)
    
    BIO12 <- mean(Precipitation_annual$x)
    BIO13 <- max(data_prec_monthly_sum$x)
    BIO14 <- min(data_prec_monthly_sum$x)
    BIO15<- sd /(1+(BIO12/12))*100
    
    

    for ( i in c(1,4,7,10)){
        prec_byquarter <- slice(data_prec_monthly_sum,i:(i+2)) %>% select(x) %>% 
                                                                    sum %>% append(prec_byquarter)
        q <- slice(data_temp_monthly_mean,i:(i+2))
        q <-mean(q$x) 
        temp_byquarter <-append(temp_byquarter,q)
    }
    df <- data.frame(temp_byquarter,prec_byquarter)
    
    wettest <- filter (df,df$prec_byquarter==max(prec_byquarter))
    dryest <- filter (df,df$prec_byquarter==min(prec_byquarter))
    warmest <- filter (df,df$temp_byquarter==max(temp_byquarter))
    coldest <- filter (df,df$temp_byquarter==min(temp_byquarter))
    
    BIO8<- wettest[,1]
    BIO9<- dryest[,1]
    BIO10<- warmest[,1]
    BIO11<- coldest[,1]
    BIO16<- wettest[,2]
    BIO17<- dryest[,2]
    BIO18<- warmest[,2]
    BIO19<- coldest[,2]
    
 
    Data_min <- openfile(path_to_data_min,time)
    Data_max <- openfile(path_to_data_max,time)


    #Calcule de la température min de tout les mois de toutes les année : 
    data_annual_monthly_min<-aggregate(Data_min$TN,by = list(vec_year,vec_month),min)

    #Calcule de la température max de tout les mois de toutes les année : 
    data_annual_monthly_max<-aggregate(Data_max$TX,by = list(vec_year,vec_month),max)
    
    data_monthly_min <- aggregate(data_annual_monthly_min$x,data_annual_monthly_min[2],mean)
    data_monthly_max <- aggregate(data_annual_monthly_max$x,data_annual_monthly_max[2],mean)
    data_monthly_range <- data_monthly_min
    data_monthly_range$max <- data_monthly_max$x
    data_monthly_range<-mutate(data_monthly_range,range = data_monthly_range$max-data_monthly_range$x)
    BIO2 <- mean(data_monthly_range$range)
    BIO5 <- max(data_monthly_max$x)
    BIO6 <- min(data_monthly_min$x)
    BIO7 <- BIO5-BIO6
    BIO3 <- (BIO2/BIO7)*100
    worldclim ( Hijmans 2005 [2] ) et sont d ́ecrites dans O’donnell 2012 [3].
    return(list(STAID,BIO1,BIO2,BIO3,BIO4,BIO5,BIO6,BIO7,BIO8,BIO9,BIO10,BIO11,BIO12,BIO13,
               BIO14,BIO15,BIO16,BIO17,BIO18,BIO19))
}

In [6]:
allBIO2 <- function (x,y) {
## Utiliser pourignorer les messages d'erreures ds savecorr 
## par ex pour avoir la liste des stations pour lesquels 
## la matrice de corrélation à pu être calculée
  out <- tryCatch(allBIO(x,y), error = function(e) NULL)
  return(out)
}

In [7]:
List_sta <- intersect(mergedata(var),Wichsta(var))

Warning message:
“ 6 failed to parse.”
Warning message:
“ 6 failed to parse.”
Warning message:
“ 6 failed to parse.”
Warning message:
“ 18 failed to parse.”


In [8]:
df <- data.frame (c('STAID','BIO1','BIO2','BIO3','BIO4','BIO5','BIO6','BIO7','BIO8','BIO9','BIO10','BIO11','BIO12','BIO13',
               'BIO14','BIO15','BIO16','BIO17','BIO18','BIO19'))

In [9]:
for ( i in 1:length(List_sta[,])){
    t <- allBIO2(List_sta[i,],time_pre)
    tt <- as.data.frame(t)
    if( length(tt)==20){df <- cbind(df,t(tt))}
}
rownames(df) <- df[,1]
colnames(df) <- df[1,]
df <- t(df[-1,-1])

In [11]:
# df       # Delete 1st ligne ?

In [12]:
write.csv(df,paste("./allBIO2020.txt",sep =''), row.names=TRUE)